Item 46 Use Descriptors for Reusable @property Methods

Things to Remember
- Reuse the behavior and validation of @property methods by defining your own descriptor classes
- Use WeakKeyDictionary to ensure that your descriptor classes don't cause memory leaks
- Don't get bogged down trying to understand exactly how \__getattribute\__ uses the descriptor protocol for getting and setting attributes 

Check the cls_sta file for more info on @classmethod and @staticmethod

Big problem with the @property decorator
- the methods it decorates can't be reused by multiple attributes of the same classes   

In [ ]:
class HomeWork:
    def __init__(self):
        self._grade = 0
    
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._grade = value

In [ ]:
# the class is easy to use
galileo = HomeWork()
galileo.grade = 95 

In [ ]:
# - it becomes tedious quickly once 
#   you have multiple attributes 
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
    
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
   
    # you need to write the boilerplate code repeatedly 
    @property
    def writing_grade(self):
       return self._writing_grade
    
    @writing_grade.setter
    def riting_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value

    @property
    def math_grade(self):
       return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self.math_grade = value


descriptor protocol
- defines how attribute access is interpreted by the language
- in our case a descriptor class can provide \__get\__ and \__set\__ methods that let you reuse the grade validation behavior without boilerplate
- in our case descriptors are better than mix-ins (Item 41) because they let you reuse the same logic for many different attributes in a single class  

In [ ]:
# the descriptor class - first attempt
class Grade:
    def __init__(self):
        self._value = 0
    
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._value = value

In [ ]:
# - refer to the cls_attr_ins_attr for more info 
#   on class attribute

class Exam:
    # class attributes
    # - a single Grade instance is shared across all
    #   Exam instances
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [ ]:
exam = Exam()
exam.writing_grade = 40

# - the exam.writing_grade = 40 statement
#   is interpreted as the following  
Exam.__dict__['writing_grade'].__set__(exam, 40)

assert exam.writing_grade == 40

# - the above statement is the same as
#   the following
assert Exam.__dict__['writing_grade'].__get__(exam, Exam) == 40

How the above code works?
- when an Exam instance doesn't have an attribute named writing_grade, Python falls back to the Exam class's attribute instead
- if this class attribute is an object that has __get__ and __set__ methods, Python assumes you want to follow the descriptor protocol 

Problem with the first attempt
- It only works if your entire program only has a single Exam instance
- This is because a single Grade instance is shared across all Exam instances

In [ ]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
# - you are changing the shared writing_grade
#   instance
second_exam.writing_grade = 75
print(f'Second {second_exam.writing_grade} is right')
print(f'First  {first_exam.writing_grade} is wrong; '
      f'should be 82')

In [ ]:
# - second attempt
# - keep track value for each Exam instance 
#   in the Grade class

class Grade:
    def __init__(self):
        self._values = {}
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        #  - save the per-instance state
        #    in a dictionary  
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._values[instance] = value

In [ ]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [ ]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print(f'First  {first_exam.writing_grade} is right')
print(f'Second {second_exam.writing_grade} is right')

Problem with the second attempt
- it leaks memory
- the _values dictionary holds a reference to every instance of Exam ever passed to \__set\__ over the lifetime of the program
- this causes instances to never have their reference count go to zero and hence can't be garbage collected

WeakKeyDictionary in weakref built-in module
- When Python runtime knows the WeakKeyDictionary is holding the instance's last remaining reference in the program it will remove the instance from the WeakKeyDictionary
- Python does the bookkeeping and ensure that the _values dictionary will be empty when all Exam instances are no longer in use

In [ ]:
from weakref import WeakKeyDictionary
# -  third attempt

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        #  - save the per-instance state
        #    in a dictionary  
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._values[instance] = value

In [ ]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [ ]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print(f'First  {first_exam.writing_grade} is right')
print(f'Second {second_exam.writing_grade} is right')